In [1]:
import sys
import os
sys.path.append(str("../src"))
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

from pathlib import Path
from dotenv import load_dotenv
load_dotenv()
PROJECT_ROOT = Path(os.getenv("PROJECT_ROOT")).resolve() # type: ignore
MODEL_ROOT = Path(os.getenv("MODEL_ROOT")).resolve() # type: ignore
DATA_ROOT = Path(os.getenv("DATA_ROOT")).resolve() # type: ignore
CONFIG_ROOT = Path(os.getenv("CONFIG_ROOT")).resolve() # type: ignore
SRC_ROOT = Path(os.getenv("SRC_ROOT")).resolve() # type: ignore

import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
dataset_path = DATA_ROOT / "IMDB" / "dataset.csv"
df = pd.read_csv(dataset_path)
display(df.head())
print(df['review'][0])

import re

def clean_html(text):
    return re.sub(r"<.*?>", " ", text)

df["review"] = df["review"].apply(clean_html)

display(df.head())

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fac

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. The filming t...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model_path = Path(str(os.getenv("MODEL_ROOT"))).resolve() / 'sft' / 'TinyLlama' / 'TinyLlama-1.1B-Chat-v1.0'
model_path = MODEL_ROOT / "base" / "EleutherAI" / "pythia-2.8b"
model_path = str(model_path)

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path).to(device)
model.warnings_issued = {}

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
# tokenize dataset and print avg. length
from tqdm.notebook import tqdm
tqdm.pandas()
tokenized_reviews = df['review'].progress_apply(lambda x: tokenizer.encode(x, return_tensors="pt", truncation=True, max_length=10000).squeeze())
lengths = tokenized_reviews.apply(len)
print(f"Average tokenized review length: {lengths.mean()}")
print(f"Max tokenized review length: {lengths.max()}")
print(f"Min tokenized review length: {lengths.min()}")

  0%|          | 0/50000 [00:00<?, ?it/s]

Average tokenized review length: 293.15688
Max tokenized review length: 3228
Min tokenized review length: 8


In [5]:
# add first 5 tokens of each review to the beginning of the review
df['prefix'] = tokenized_reviews.apply(lambda x: tokenizer.decode(x[0:5]))
display(df[['prefix', 'review']].head())

,prefix,review
0,One of the other reviewers,One of the other reviewers has mentioned that ...
1,A wonderful little production.,A wonderful little production. The filming t...
2,I thought this was a,I thought this was a wonderful way to spend ti...
3,Basically there's a family,Basically there's a family where a little boy ...
4,Petter Mattei,"Petter Mattei's ""Love in the Time of Money"" is..."


In [6]:
# dataset split
# train: 80%, val: 10%, test: 10%
train_size = int(0.8 * len(df))
val_size = int(0.1 * len(df))
test_size = len(df) - train_size - val_size

df = df.sample(frac=1, random_state=42).reset_index(drop=True)  # shuffle the dataset

train_df = df.iloc[:train_size]
val_df = df.iloc[train_size:train_size + val_size]
test_df = df.iloc[train_size + val_size:]

print(f"Train size: {len(train_df)}, Val size: {len(val_df)}, Test size: {len(test_df)}")

# save split datasets
train_df.to_csv(DATA_ROOT / "IMDB" / "train.csv", index=False)
val_df.to_csv(DATA_ROOT / "IMDB" / "val.csv", index=False)
test_df.to_csv(DATA_ROOT / "IMDB" / "test.csv", index=False)

Train size: 40000, Val size: 5000, Test size: 5000
